In [16]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
# Load the dataset
client = MongoClient("localhost", 27017)
db = client["IF29"]
collec = db.user_label
data = pd.DataFrame(list(collec.find()))

In [18]:
# Define features and target variable
data_removed = data[data["label"] != 0]

# Define features and target variable
features = [
    "verified","protected","friend_nb","listed_nb","follower_nb","favorites_nb","len_description","hash_avg","mention_avg","url_avg","symbols_avg","tweet_nb","tweet_user_count","user_lifetime","tweet_frequency","friend_frequency","aggressivity","visibility","ff_ratio"
]
X = data_removed[features]
y = data_removed["label"]

y_loc = y.iloc[:20000]
X_loc = X.iloc[:20000]

In [19]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_loc, y_loc, test_size=0.3, random_state=42)

In [20]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
# Create an SVM classifier
svm_classifier = SVC(kernel="rbf")

In [22]:
# Train the classifier
svm_classifier.fit(X_train, y_train)

SVC()

In [23]:
# Make predictions on the test data
y_pred = svm_classifier.predict(X_test)

In [24]:
# Nombre de vecteurs supports
num_support_vectors = svm_classifier.n_support_
print("Nombre de vecteurs supports par classe :", num_support_vectors)


Nombre de vecteurs supports par classe : [ 51 257]


In [25]:
# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2969    2]
 [   0 3029]]
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      2971
           1       1.00      1.00      1.00      3029

    accuracy                           1.00      6000
   macro avg       1.00      1.00      1.00      6000
weighted avg       1.00      1.00      1.00      6000



In [26]:
# Générer la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion avec seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Prédictions')
plt.ylabel('Vraies valeurs')
plt.title('Matrice de Confusion')
plt.show()

In [27]:
# Calculer les valeurs TP, FP, FN, TN
TN, FP, FN, TP = conf_matrix.ravel()
print(f'Vrais Négatifs (TN): {TN}')
print(f'Faux Positifs (FP): {FP}')
print(f'Faux Négatifs (FN): {FN}')
print(f'Vrais Positifs (TP): {TP}')

Vrais Négatifs (TN): 2969
Faux Positifs (FP): 2
Faux Négatifs (FN): 0
Vrais Positifs (TP): 3029


In [28]:
import matplotlib.pyplot as plt

# Préparer les données pour la prédiction
X_full = data[features]
X_full = scaler.transform(X_full)  # Normaliser les données

# Prédire les labels avec le modèle SVM
y_pred_full = svm_classifier.predict(X_full)

# Ajouter les labels prédits au DataFrame
data["predicted_label"] = y_pred_full

# Créer un graphique de visualisation
plt.figure(figsize=(12, 8))
scatter = plt.scatter(data["visibility"], data["aggressivity"], c=data["predicted_label"], cmap='viridis', alpha=0.5)
plt.xlabel("Visibility")
plt.ylabel("Aggressivity")
plt.title("Predicted Labels based on Visibility and Aggressivity")
plt.legend(*scatter.legend_elements(), title="Predicted Label")
plt.grid(True)
plt.show()



In [29]:
cluster_describe = data.groupby('predicted_label').describe()
print(cluster_describe)

                       _id                                                   \
                     count          mean           std    min           25%   
predicted_label                                                               
-1               1297901.0  2.023318e+17  3.717185e+17  730.0  2.924679e+08   
 1                545538.0  2.878774e+17  4.202157e+17  418.0  3.403433e+08   

                                                            verified  \
                          50%           75%           max      count   
predicted_label                                                        
-1               1.246178e+09  4.340409e+09  1.008391e+18  1297901.0   
 1               2.295805e+09  8.051137e+17  1.008393e+18   545538.0   

                           ...  ff_ratio                label            \
                     mean  ...       75%       max      count      mean   
predicted_label            ...                                            
-1               0

In [30]:
# Enregistrer le DataFrame en fichier CSV
cluster_describe.to_csv('SVM_describe.csv')